### Import necessary modules

In [1]:
import torch
from torchvision.transforms import transforms
from models import *
from constants import *
from sklearn.metrics import roc_auc_score
from segmentation_inference import *
from metrics import *
from PIL import Image
from datasets import *
from torch.utils.data import DataLoader
from custom_transformations import HistogramEqualization
from util import *


### Create Dataset and Data Loader

In [2]:
transformation_list = [
    transforms.Resize((256, 256)),
    HistogramEqualization(),
    transforms.ToTensor(),
]
image_transformation = transforms.Compose(transformation_list)

test_dataset = CheXpertDataset(data_path="data/CheXpert-v1.0-small/valid.csv",
                               uncertainty_policy="zeros", transform=image_transformation)

test_dataloader = DataLoader(dataset=test_dataset, batch_size=1, shuffle=True)


### Load Models

In [3]:
%%capture
heart_model_path = "models/heart_segmentation/ext_augmentation_lr=0.0001_batch=3_24.3_13:11.pth"
lung_model_path = "models/lung_segmentation/ext_augmentation_lr=0.0001_batch=3_24.3_10:44.pth"
heart_model = load_segmentation_model(heart_model_path, device="cpu")
lung_model = load_segmentation_model(lung_model_path, device="cpu")

### Calculate CTR for CheXpert images

In [4]:
ground_truth = torch.FloatTensor()
ctr = torch.FloatTensor()
prediction = torch.FloatTensor()
for step, (image, label) in enumerate(test_dataloader):
    print(step)
    ground_truth = torch.cat((ground_truth, label), 0)
    ctr_for_image = torch.tensor([ctr_from_tensor(image, heart_model, lung_model)])
    torch.cat((ctr, ctr_for_image), 0)
    prediction_for_image = torch.ones(1) if ctr_for_image > 0.5 else torch.zeros(1)
    prediction = torch.cat((prediction, prediction_for_image), 0)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201


### Calculate AUROC score

In [6]:
score = roc_auc_score(ground_truth.squeeze(), prediction)
print(score)

0.5737522281639929
